<font size=6><b>Lec09. 교차검증
* ref : https://amueller.github.io/aml/04-model-evaluation/1-data-splitting-strategies.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------------------------------
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier
# ------------------------------------------------------

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score 
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import roc_curve, roc_auc_score
# ------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder


import warnings
warnings.filterwarnings(action="ignore")

In [2]:
# ---------------------- 차트 관련 속성 (한글처리, 그리드) --------------
plt.rcParams['font.family'] = "Malgun Gothic"
plt.rcParams['axes.unicode_minus'] = False

# Data Load
* df.info()
* df.shape
* df.head()

In [3]:
data = load_iris()
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
df = pd.DataFrame(data=data["data"], columns=data["feature_names"] )  #index=[]
df.columns = ["sl","sw","pl","pw"]
df['target'] = data["target"]
df.head()

,sl,sw,pl,pw,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sl      150 non-null    float64
 1   sw      150 non-null    float64
 2   pl      150 non-null    float64
 3   pw      150 non-null    float64
 4   target  150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB


In [6]:
df.shape

(150, 5)

# 교차검증(cross-validation)
* Defaults in scikit-learn
  * 5-fold in 0.22
  * For classification cross-validation is stratified  * train_test_split has stratify option: train_test_split(X, y, stratify=y)
  * 
No shuffle by default!

* enumerate 연습

In [7]:
for v in [1,2,3]:
    print(v)

1
2
3


In [8]:
for loop_cnt, v in enumerate([1,2,3]):
    print(loop_cnt, v)

0 1
1 2
2 3


In [9]:
for loop_cnt, (v1, v2) in enumerate([[1,10], [2,20], [3,30]]):
    print(loop_cnt, v1, v2)

0 1 10
1 2 20
2 3 30


## KFold()
* 회귀모델

In [10]:
from sklearn.model_selection import train_test_split, KFold

```python
class sklearn.model_selection.KFold(n_splits=5, *, shuffle=False, random_state=None)
```

In [11]:
df['target'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [12]:
X = df.drop('target', axis=1)
y = df['target']

score_list = []
kfold = KFold(n_splits=10, shuffle=True, random_state=33)
for i, (train_idx, test_idx) in enumerate(kfold.split(X)):
    # print(i, train_idx, test_idx)
    print(df.loc[train_idx, 'target'].values)
    print(df.loc[test_idx, 'target'].values)

    # print(y.loc[train_idx].values)
    # print(y.loc[test_idx].values)
    # print(X.loc[train_idx].values)
    # print(X.loc[test_idx].values)
    y_train, y_test = y.loc[train_idx], y.loc[test_idx]
    X_train, X_test = X.loc[train_idx], X.loc[test_idx]
    print(i, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
    model.fit(X_train, y_train)
    pred = model.predict( X_test)
    f1   = f1_score(y_test, pred, average="macro")

    score_list.append(f1)
    print(f"f1 : {f1:.4f}")
    print("---"*30)

print(f"평균점수 : {np.mean(score_list):.4f}")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 1 1 1 1 2 2 2 2 2 2 2]
0 (135, 4) (15, 4) (135,) (15,)
f1 : 1.0000
------------------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 1 1 1 1 2 2 2 2 2 2 2]
1 (135, 4) (15, 4) (135,) (15,)
f1 : 0.8182
------------------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 

## Stratified KFold
* 분류 모델
* 학습 시 target 비율에 맞게 각 fold에 넣어준다

```python
class sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)
```

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
X = df.drop('target', axis=1)
y = df['target']

score_list = []
# kfold = KFold(n_splits=10, shuffle=True, random_state=33)
# for i, (train_idx, test_idx) in enumerate(kfold.split(X)):

skfold = StratifiedKFold(n_splits=3, shuffle =True, random_state=33)
for i, (train_idx, test_idx) in enumerate(skfold.split(X,y)):
    # print(i, train_idx, test_idx)
    print(df.loc[train_idx, 'target'].values)
    # print(df.loc[test_idx, 'target'].values)
    print(y[train_idx].value_counts())
    # print(y.loc[train_idx].values)
    # print(y.loc[test_idx].values)
    y_train, y_test = y.loc[train_idx], y.loc[test_idx]
    # print(X.loc[train_idx].values)
    # print(X.loc[test_idx].values)
    X_train, X_test = X.loc[train_idx], X.loc[test_idx]
    
    # print(i, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
    model.fit(X_train, y_train)
    pred = model.predict( X_test)
    f1   = f1_score(y_test, pred, average="macro")
    print(f"f1 : {f1:.4f}")

    score_list.append(f1)
    print("---"*30)

print(f"평균점수 : {np.mean(score_list):.4f}")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
2    34
0    33
1    33
Name: target, dtype: int64
f1 : 0.9392
------------------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
1    34
0    33
2    33
Name: target, dtype: int64
f1 : 0.9798
------------------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
0    34
1    33
2    33
Name: target, dtype: int64
f1 : 0.9608
-----------------

### train_test_split(stratify=타겟피쳐 이름)
* stratify 속성으로 fold에 골고루 넣어준다
* stratify에 y값을 넣으면 stratifiedfold()와 동일한 효과!

```python
sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
```

In [15]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_train.value_counts())

(100, 4) (50, 4) (100,) (50,)
0    34
1    33
2    33
Name: target, dtype: int64


## Cross_val_score()

```python
sklearn.model_selection.cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
```

In [16]:
from sklearn.model_selection import cross_val_score

In [17]:
model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
X = df.drop('target', axis=1)
y = df['target']


# ------------------------------------
# cv :(Stratified)KFold
# ------------------------------------
skfold = StratifiedKFold(n_splits=3, shuffle =True, random_state=33)       # 분류
kfold = KFold(n_splits=3, shuffle=True, random_state=33)                   # 회귀

score_arr = cross_val_score(model, X, y, scoring='f1_macro', cv=kfold)
print(score_arr)
print(f"평균점수 : {score_arr.mean():.4f}")

[0.92592593 0.93297297 0.93560976]
평균점수 : 0.9315


## Cross-validate()
* cross_val_score() : 점수를 array로 준다
* cross_validate() : 딕셔너리로 준다(점수, 실행시간, 기타...)

```python
sklearn.model_selection.cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, return_indices=False, error_score=nan)
```

In [18]:
from sklearn.model_selection import cross_validate

In [19]:
model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
X = df.drop('target', axis=1)
y = df['target']


# ------------------------------------
# cv :(Stratified)KFold
# ------------------------------------
skfold = StratifiedKFold(n_splits=3, shuffle =True, random_state=33)       # 분류
kfold = KFold(n_splits=3, shuffle=True, random_state=33)                  # 회귀

score_dict = cross_validate(model, X, y, scoring='f1_macro', cv=kfold)
print(score_dict)
# print(f"평균점수 : {score_arr.mean():.4f}")

print(pd.DataFrame(score_dict))
print(f"평균점수 : {score_dict['test_score']}")

pd.DataFrame(score_dict).head()

{'fit_time': array([0., 0., 0.]), 'score_time': array([0., 0., 0.]), 'test_score': array([0.92592593, 0.93297297, 0.93560976])}
   fit_time  score_time  test_score
0       0.0         0.0    0.925926
1       0.0         0.0    0.932973
2       0.0         0.0    0.935610
평균점수 : [0.92592593 0.93297297 0.93560976]


,fit_time,score_time,test_score
0,0.0,0.0,0.925926
1,0.0,0.0,0.932973
2,0.0,0.0,0.935610


# GridSearchCV
* 교차검증(cross-validation) + 튜닝 (hyper parameter)

```python
class sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)
```

In [20]:
from sklearn.model_selection import GridSearchCV

model = DecisionTreeClassifier(random_state=111, min_samples_split=2)
X = df.drop('target', axis=1)
y = df['target']
# max_depth=None, min_samples_split=2, min_samples_leaf=1
myparam = {"max_depth" : [10,20,30],                 # myparam에 기입된 값만 학습에 활용
          "min_samples_split" : [1,2,3,4],
          "min_samples_leaf" : [1,2,3]
          }

skfold = StratifiedKFold(n_splits=3, shuffle =True, random_state=33)
gcv_model = GridSearchCV(model, param_grid=myparam, scoring="f1_macro", refit=True, cv = skfold)
gcv_model.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=33, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=111),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3, 4]},
             scoring='f1_macro')

In [21]:
# gcv_model.cv_results_
print( gcv_model.best_estimator_ )
print( gcv_model.best_score_ )
print( gcv_model.best_params_ )

DecisionTreeClassifier(max_depth=10, min_samples_leaf=3, random_state=111)
0.973243994567524
{'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 2}


In [22]:
# refit=False
# dt = DecisionTreeClassifier(max_depth=10, min_samples_split=3, min_samples_leaf=4)
# dt.fit()

# refit=True
# gcv_model.fit()
